# Лабораторная работа №3
## Выполнил студент группы БВТ2005 Макбари Даниш Александрович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [2]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [72]:
def prefix(pattern):
    pi = [0 for i in range(len(pattern))]
    j = 0
    i = 1
    while i < len(pi):
        if pattern[j] == pattern[i]:
            pi[i] = j + 1
            i += 1
            j += 1
        elif j == 0:
            pi[i] = 0
            i += 1
        else:
            j = pi[j - 1]
    return pi

def kmp(string, pattern, case_flag):
    if case_flag:
        string = string.lower()
        pattern = pattern.lower()
    matches = []
    pi = prefix(pattern)
    i = 0
    j = 0
    while i < len(string):
        if string[i] == pattern[j]:
            i += 1
            j += 1 
            if j == len(pattern):
                matches.append(i - j)
                j = pi[j - 1]
        else:
            if j > 0:
                j = pi[j - 1]
            else:
                i += 1
    return matches

print("string: ", end = '')
string = input()
print("pattern: ", end = '')
pattern = input()
print(f"prefix table: {prefix(pattern)}")
t1 = t.now()
result = kmp(string, pattern, True)
t2 = t.now() - t1
print(f"matches: {result}")
print(f"time: {t2}")

string: oniOnions plonionso
pattern: onions
prefix table: [0, 0, 0, 1, 2, 0]
matches: [3, 12]
time: 0:00:00.000071


#### Упрощенный алгоритм Бойера-Мура

In [70]:
def shift(pattern):
    table = defaultdict(lambda: len(pattern))
    for i in range(len(pattern) - 1):
        table[pattern[i]] = len(pattern) - 1 - i
    return table

def bmh(string, pattern, case_flag):
    if case_flag:
        string = string.lower()
        pattern = pattern.lower()
    matches = []
    table = shift(pattern)
    i = len(pattern) - 1
    while i < len(string):
        k = 0
        while k < len(pattern) and string[i - k] == pattern[len(pattern) - 1 - k]:
            k += 1
        if k != len(pattern):
            i += table[string[i]]
        else:
            matches.append(i - len(pattern) + 1)
            i += table[string[i]]
    return matches

print("string: ", end = '')
string = input()
print("pattern: ", end = '')
pattern = input()
t1 = t.now()
result = bmh(string, pattern, True)
t2 = t.now() - t1
print(f"matches: {result}")
print(f"time: {t2}")

string: oniOnions plonionso
pattern: onions
matches: [3, 12]
time: 0:00:00.000079


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [75]:
import copy

class board:
    def __init__(self, arr, g, parent):
        self.board = arr
        self.g = g
        self.h = 0
        self.parent = parent
        for i in range(len(self.board)):
            for j in range(len(self.board)):
                if self.board[i][j] == 0:
                    self.zeroX = i
                    self.zeroY = j
                    continue
                if i * len(self.board) + j + 1 != self.board[i][j]:
                    self.h += 1
        self.f = self.g + self.h
    
    def print(self):
        for i in range(len(self.board)):
            for j in range(len(self.board)):
                print(self.board[i][j], end = ' ')
            print()
        print() 
    
    def get_neighbors(self):
        board_list = []
        if self.zeroX > 0:
            arr = copy.deepcopy(self.board)
            arr[self.zeroX][self.zeroY] = arr[self.zeroX - 1][self.zeroY]
            arr[self.zeroX - 1][self.zeroY] = 0
            board_list.append(board(arr, self.g + 1, self))
        if self.zeroX < len(self.board) - 1:
            arr = copy.deepcopy(self.board)
            arr[self.zeroX][self.zeroY] = arr[self.zeroX + 1][self.zeroY]
            arr[self.zeroX + 1][self.zeroY] = 0
            board_list.append(board(arr, self.g + 1, self))
        if self.zeroY > 0:
            arr = copy.deepcopy(self.board)
            arr[self.zeroX][self.zeroY] = arr[self.zeroX][self.zeroY - 1]
            arr[self.zeroX][self.zeroY - 1] = 0
            board_list.append(board(arr, self.g + 1, self))
        if self.zeroY < len(self.board) - 1:
            arr = copy.deepcopy(self.board)
            arr[self.zeroX][self.zeroY] = arr[self.zeroX][self.zeroY + 1]
            arr[self.zeroX][self.zeroY + 1] = 0
            board_list.append(board(arr, self.g + 1, self))
        return board_list

def add_to_open(open, neighbor):
    for node in open:
        if neighbor.board == node.board and neighbor.f >= node.f:
            return False
    return True

def a_star(start):
    open = [start]
    closed = []
    
    while len(open) > 0:
        open = sorted(open, key = lambda board: board.f)
        current = open.pop(0)
        closed.append(current)
        
        if current.h == 0:
            path = []
            while current != None:
                path.insert(0, current)
                current = current.parent
            for b in path:
                b.print()
            return
        
        neighbors = current.get_neighbors()
        for neighbor in neighbors:
            if neighbor in closed:
                continue
            
            if add_to_open(open, neighbor):
                open.append(neighbor)
    return

def solve(arr):
    b = board(arr, 0, None)
    a_star(b)

t1 = t.now()
solve([[0, 3, 5], [8, 6, 7], [1, 2, 4]])
t2 = t.now() - t1
print(f"time: {t2}")

0 3 5 
8 6 7 
1 2 4 

8 3 5 
0 6 7 
1 2 4 

8 3 5 
1 6 7 
0 2 4 

8 3 5 
1 6 7 
2 0 4 

8 3 5 
1 6 7 
2 4 0 

8 3 5 
1 6 0 
2 4 7 

8 3 5 
1 0 6 
2 4 7 

8 3 5 
0 1 6 
2 4 7 

8 3 5 
2 1 6 
0 4 7 

8 3 5 
2 1 6 
4 0 7 

8 3 5 
2 1 6 
4 7 0 

8 3 5 
2 1 0 
4 7 6 

8 3 0 
2 1 5 
4 7 6 

8 0 3 
2 1 5 
4 7 6 

8 1 3 
2 0 5 
4 7 6 

8 1 3 
0 2 5 
4 7 6 

0 1 3 
8 2 5 
4 7 6 

1 0 3 
8 2 5 
4 7 6 

1 2 3 
8 0 5 
4 7 6 

1 2 3 
0 8 5 
4 7 6 

1 2 3 
4 8 5 
0 7 6 

1 2 3 
4 8 5 
7 0 6 

1 2 3 
4 0 5 
7 8 6 

1 2 3 
4 5 0 
7 8 6 

1 2 3 
4 5 6 
7 8 0 

time: 0:01:52.883001
